In [1]:
import pandas as pd
import numpy as np
import warnings
#Let's do text cleaansing now to pre process the data for topic modelling
import re
from pprint import pprint
#
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
#
# spacy for lemmatization
import spacy
# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline
#Cleaning comment section
import nltk
# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)
warnings.filterwarnings("ignore",category=DeprecationWarning)

C:\ProgramData\Anaconda3\lib\site-packages\past\types\oldstr.py:33: DeprecationWarning: invalid escape sequence \d
  """
C:\ProgramData\Anaconda3\lib\site-packages\nose\config.py:178: DeprecationWarning: invalid escape sequence \.
  """
C:\ProgramData\Anaconda3\lib\site-packages\nltk\decorators.py:68: DeprecationWarning: `formatargspec` is deprecated since Python 3.5. Use `signature` and the `Signature` object directly
  regargs, varargs, varkwargs, defaults, formatvalue=lambda value: ""
C:\ProgramData\Anaconda3\lib\site-packages\nltk\featstruct.py:1296: DeprecationWarning: invalid escape sequence \d
  name, n = re.sub('\d+$', '', var.name), 2
C:\ProgramData\Anaconda3\lib\site-packages\nltk\featstruct.py:2092: DeprecationWarning: invalid escape sequence \d
  RANGE_RE = re.compile('(-?\d+):(-?\d+)')
C:\ProgramData\Anaconda3\lib\site-packages\nltk\sem\evaluate.py:311: DeprecationWarning: invalid escape sequence \ 
  """
C:\ProgramData\Anaconda3\lib\site-packages\nltk\sem\relextract.py:13

C:\ProgramData\Anaconda3\lib\site-packages\nltk\inference\discourse.py:44: DeprecationWarning: invalid escape sequence \ 
  """
C:\ProgramData\Anaconda3\lib\site-packages\nltk\stem\lancaster.py:195: DeprecationWarning: invalid escape sequence \*
  valid_rule = re.compile("^[a-z]+\*?\d[a-z]*[>\.]?$")
C:\ProgramData\Anaconda3\lib\site-packages\nltk\stem\lancaster.py:228: DeprecationWarning: invalid escape sequence \*
  valid_rule = re.compile("^([a-z]+)(\*?)(\d)([a-z]*)([>\.]?)$")
C:\ProgramData\Anaconda3\lib\site-packages\nltk\stem\porter.py:181: DeprecationWarning: invalid escape sequence \m
  """
C:\ProgramData\Anaconda3\lib\site-packages\nltk\corpus\__init__.py:116: DeprecationWarning: invalid escape sequence \.
  '.*\.(test|train).*',
C:\ProgramData\Anaconda3\lib\site-packages\nltk\corpus\__init__.py:123: DeprecationWarning: invalid escape sequence \.
  '.*\.(test|train).*',
C:\ProgramData\Anaconda3\lib\site-packages\nltk\corpus\__init__.py:126: DeprecationWarning: invalid escape se

In [2]:
# Read recipe inputs
clean_data_topic_model = pd.read_csv('clean_data_topic_model.csv')

In [3]:
with open('stopwords.txt') as f:
    stopwords = f.read()

In [4]:
clean_data_topic_model.head(2)

,case_number,case_summary,lang_info,case_created_date,case_typ,case_injury_typ,case_txt,country_nme,case_supervisor_nme,case_root_cause_txt,...,case_report_year,case_level_severity,primary_business,primary_business_segment,site_location_nme,region,case_followup_flag,month,month_year,quarter
0,9631,While removing root grinding machine clamps su...,en,2019-01-02T00:00:00.000Z,Injury,"Contusion, Bruise",While removing root grinding machine clamps su...,India,C H Veera Babu,When operator lowering Bush grinding machine (...,...,2019,Level D,GE Renewable Energy,LM Wind Power,Dabaspet,ASEAN/India/JP/KR,Yes,1,2019-1,Q1-2019
1,9632,Electrician was removing the insulation of arm...,en,2019-01-02T00:00:00.000Z,Injury,"Laceration, Cut, Open Wound",Electrician was removing the insulation of arm...,India,SHISHU PAL SINGH,Electrician choose wrong position,...,2019,Level D,GE Renewable Energy,Hydro,Hydro - INDIA - OM and Services,ASEAN/India/JP/KR,Yes,1,2019-1,Q1-2019


In [5]:
df= clean_data_topic_model.copy()

In [6]:
def text_cleaner(text,num):
    newString = text.lower()
    newString = re.sub(r'\([^)]*\)', '', newString)
    newString = re.sub(r"[0-9]", "", newString)
    newString = re.sub(',',' ', newString)
    newString = re.sub('-',' ', newString)
    newString = re.sub('_',' ', newString)
    newString = re.sub('&','and', newString)
    newString = re.sub('[()]', '', newString)
    newString = re.sub(';',' ', newString)
    newString = re.sub(r"'s\b","",newString)
    newString = re.sub("[^a-zA-Z]", " ", newString)
    newString = re.sub(r"Ä¢", "", newString)
    newString = re.sub(r"¬∑", "", newString)
    newString = re.sub(r"\'", "", newString)
    newString = re.sub(r"\"", "", newString)
    newString = re.sub(r"\n", "", newString)
    newString = re.sub(r"\r", "", newString)
    return newString.strip().lower()

In [7]:
X = []
for t in df['case_summary']:
    X.append(text_cleaner(t,0))

In [8]:
#Soting list in the column back again
df['case_summary'] = pd.DataFrame(X)
df['case_summary'].head()

0    while removing root grinding machine clamps su...
1    electrician was removing the insulation of arm...
2    employee strained his back while performing a ...
3    victim was working at   downwind for combi lay...
4    kapamada web indirme s ras nda axial guide  n ...
Name: case_summary, dtype: object

In [9]:
def converter(x):
    try:
        return ' '.join([x.lower() for x in str(x).split() if x not in stopwords])
    except AttributeError:
        return None  # or some other value

df['case_summary'] = df['case_summary'].apply(converter)
df.head(2)

,case_number,case_summary,lang_info,case_created_date,case_typ,case_injury_typ,case_txt,country_nme,case_supervisor_nme,case_root_cause_txt,...,case_report_year,case_level_severity,primary_business,primary_business_segment,site_location_nme,region,case_followup_flag,month,month_year,quarter
0,9631,removing root grinding machine clamps sustaine...,en,2019-01-02T00:00:00.000Z,Injury,"Contusion, Bruise",While removing root grinding machine clamps su...,India,C H Veera Babu,When operator lowering Bush grinding machine (...,...,2019,Level D,GE Renewable Energy,LM Wind Power,Dabaspet,ASEAN/India/JP/KR,Yes,1,2019-1,Q1-2019
1,9632,electrician removing insulation armored cable ...,en,2019-01-02T00:00:00.000Z,Injury,"Laceration, Cut, Open Wound",Electrician was removing the insulation of arm...,India,SHISHU PAL SINGH,Electrician choose wrong position,...,2019,Level D,GE Renewable Energy,Hydro,Hydro - INDIA - OM and Services,ASEAN/India/JP/KR,Yes,1,2019-1,Q1-2019


In [10]:
#Tokenise
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(df.case_summary))

print(data_words[:4])

[['removing', 'root', 'grinding', 'machine', 'clamps', 'sustained', 'injury', 'finger'], ['electrician', 'removing', 'insulation', 'armored', 'cable', 'knife', 'ip', 'supporting', 'electrician', 'holding', 'cable', 'removal', 'insulation', 'knife', 'contact', 'ip', 'left', 'hand', 'middle', 'finger', 'causing', 'minor', 'cut', 'injury', 'wearing', 'cut', 'resistant', 'gloves', 'contact', 'rear', 'glove', 'ineffective', 'preventing', 'injury', 'contact', 'happened', 'knife', 'stationary', 'extent', 'injury', 'limited', 'incident', 'happened', 'hrs', 'ip', 'resumed'], ['employee', 'strained', 'performing', 'person', 'lift', 'roll', 'fiberglass', 'rolled', 'wind', 'shell', 'mold'], ['victim', 'downwind', 'combi', 'layer', 'application', 'performing', 'activity', 'handling', 'resin', 'chemical', 'handling', 'resin', 'chemical', 'chemical', 'spalsh', 'eyes', 'worn', 'breathe', 'easy', 'mask', 'properly']]


In [11]:
import re, nltk, spacy, gensim
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append(" ".join([token.lemma_ if token.lemma_ not in ['-PRON-'] else '' for token in doc if token.pos_ in allowed_postags]))
    return texts_out

In [12]:
# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
spacy.load('en')
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
data_lemmatized = lemmatization(data_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
print(data_lemmatized[:2])

['remove root grind machine clamp sustain injury finger', 'remove insulation armor cable knife support cable removal insulation knife contact ip left hand middle finger cause minor cut injury wear cut resistant glove contact rear glove ineffective prevent injury contact happen knife stationary extent injury limit incident happen hrs resume']


In [13]:
# Sklearn
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV

In [14]:
vectorizer = CountVectorizer(analyzer='word',
                             min_df=5,                          #min_df is used for removing terms that appear too infrequently
                             max_df=0.20,                          #max_df is used for removing terms that appear too frequently, also known as "corpus-specific stop words".
#                              ngram_range=(1,2),
                             token_pattern='[a-zA-Z0-9]{3,}',      # num chars > 3
#                              stop_words = stopwords
#                              stop_words='english',             # remove stop words, already removed
#                              lowercase=True,                   # already converte to lower case
                             # max_features=50000,             # max number of uniq words
                            )

In [15]:
data_vectorized = vectorizer.fit_transform(data_lemmatized)

In [16]:
word_list = vectorizer.get_feature_names()
count_list = data_vectorized.toarray().sum(axis=0)

In [17]:
print(dict(zip(word_list,count_list)))

{'abnormal': 5, 'abnormality': 12, 'abrasion': 34, 'access': 29, 'accident': 33, 'accidentally': 34, 'accompany': 17, 'accord': 11, 'acetone': 10, 'action': 16, 'activate': 8, 'active': 19, 'activity': 113, 'additional': 7, 'adhesive': 6, 'adjacent': 6, 'adjust': 29, 'administer': 8, 'advice': 8, 'advise': 16, 'affect': 9, 'afin': 5, 'afternoon': 16, 'afterwards': 9, 'aider': 12, 'air': 87, 'alcohol': 7, 'align': 9, 'alignment': 11, 'allege': 6, 'allow': 9, 'ambulance': 9, 'amount': 6, 'angle': 7, 'ankle': 125, 'appear': 15, 'application': 13, 'applied': 5, 'apply': 70, 'appointment': 9, 'approach': 5, 'approx': 5, 'approximately': 37, 'ara': 6, 'arm': 171, 'arpt': 8, 'arrange': 10, 'arrive': 20, 'assemble': 22, 'assembly': 23, 'assessment': 8, 'assign': 17, 'assist': 10, 'assistance': 10, 'attach': 25, 'attachment': 6, 'attack': 6, 'attempt': 41, 'attend': 26, 'attention': 12, 'avec': 10, 'avoid': 14, 'aware': 15, 'backwards': 22, 'bad': 8, 'bag': 41, 'balance': 24, 'band': 29, 'banda

In [18]:
# Plotting tools
import pyLDAvis
import pyLDAvis.sklearn
import matplotlib.pyplot as plt
%matplotlib inline

In [19]:
# Build LDA Model
fixed_seed = 100
lda_model = LatentDirichletAllocation(n_components=10,               # Number of topics
                                      max_iter=10,               # Max learning iterations
                                      learning_method='online',
                                      random_state=np.random.RandomState(fixed_seed),          # Random state
                                      batch_size=128,            # n docs in each learning iter
                                      evaluate_every = -1,       # compute perplexity every n iters, default: Don't
                                      n_jobs = -1,               # Use all available CPUs
                                     )
lda_output = lda_model.fit_transform(data_vectorized)

print(lda_model)  # Model attributes

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='online', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=10, n_jobs=-1,
                          perp_tol=0.1,
                          random_state=<mtrand.RandomState object at 0x00000246AB745630>,
                          topic_word_prior=None, total_samples=1000000.0,
                          verbose=0)


In [20]:
# column names
topicnames = ["Topic" + str(i) for i in range(lda_model.n_components)]

In [21]:
# index names
docnames = ["Doc" + str(i) for i in range(len(df.case_summary))]

In [22]:
# Make the pandas dataframe
df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)

In [23]:
# Get dominant topic for each document
dominant_topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['dominant_topic'] = dominant_topic

In [24]:
# Styling
def color_green(val):
    color = 'green' if val > .1 else 'black'
    return 'color: {col}'.format(col=color)

def make_bold(val):
    weight = 700 if val > .1 else 400
    return 'font-weight: {weight}'.format(weight=weight)

In [25]:
# Apply Style
df_document_topics = df_document_topic.head().style.applymap(color_green).applymap(make_bold)
df_document_topics

,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,Topic6,Topic7,Topic8,Topic9,dominant_topic
Doc0,0.380000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.530000,9
Doc1,0.000000,0.000000,0.000000,0.000000,0.000000,0.680000,0.000000,0.020000,0.000000,0.280000,5
Doc2,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.900000,0.010000,8
Doc3,0.530000,0.330000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.090000,0
Doc4,0.050000,0.550000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,1


In [26]:
df_document_topic=df_document_topic.loc[:,['dominant_topic']]
df_document_topic1= df_document_topic.reset_index()
df_document_topic1.head(10)

,index,dominant_topic
0,Doc0,9
1,Doc1,5
2,Doc2,8
3,Doc3,0
4,Doc4,1
5,Doc5,5
6,Doc6,3
7,Doc7,5
8,Doc8,6
9,Doc9,6


In [27]:
df_document_topic1.shape

(2439, 2)

In [28]:
df_topic_distribution = df_document_topic1['dominant_topic'].value_counts().reset_index(name="Num Documents")
df_topic_distribution.columns = ['Topic Num', 'Num Documents']
df_topic_distribution

,Topic Num,Num Documents
0,5,425
1,0,389
2,9,360
3,6,356
4,8,350
5,7,256
6,1,84
7,4,83
8,3,82
9,2,54


In [29]:
# # Visulaiise the topic keywords
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(lda_model, data_vectorized, vectorizer, mds='tsne')
panel

PreparedData(topic_coordinates=               x          y  topics  cluster       Freq
topic                                                  
9      33.543354  12.615475       1        1  15.563040
5       1.785664  23.311750       2        1  15.532498
7     -54.906109  -1.659932       3        1  14.468005
6     -30.471607  26.496729       4        1  14.017382
8      12.781395 -19.880081       5        1  12.652709
0     -60.720341  39.444664       6        1   8.257946
1      16.416386  53.377968       7        1   5.675728
3     -29.074268 -33.623638       8        1   4.945778
4     -17.079494  -2.989667       9        1   4.641708
2     -21.786774  59.895309      10        1   4.245207, topic_info=         Term        Freq       Total Category  logprob  loglift
322    finger  481.000000  481.000000  Default  30.0000  30.0000
590      pain  332.000000  332.000000  Default  29.0000  29.0000
219       cut  407.000000  407.000000  Default  28.0000  28.0000
44        arm  194.000000  194.000000  Default  27.0000  27.0000
381      hand  531.000000  531.000000  Default  26.0000  26.0000
..        ...         ...         ...      ...      ...      ...
353     front   18.656143   41.524722  Topic10  -4.2128   2.3593
416       ice   22.321491   76.235606  Topic10  -4.0334   1.9311
552      move   21.767110  194.757060  Topic10  -4.0586   0.9680
341   forearm   14.991374   35.539166  Topic10  -4.4315   2.2962
642  position   14.004840   84.349263  Topic10  -4.4996   1.3638

[568 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
1         3  0.910124  abnormality
2         1  0.786562     abrasion
2         4  0.211767     abrasion
3         8  0.984615       access
4         3  0.382802     accident
...     ...       ...          ...
1010      1  0.916227       wrench
1010      6  0.018699       wrench
1010      7  0.037397       wrench
1011      1  0.774479        wrist
1011      5  0.214471        wrist

[885 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[10, 6, 8, 7, 9, 1, 2, 4, 5, 3])

In [30]:
pyLDAvis.save_html(panel, 'lda.html')

In [31]:
def show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=20):
    keywords = np.array(vectorizer.get_feature_names())
    topic_keywords = []
    for topic_weights in lda_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords

topic_keywords = show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=20)

In [32]:
df_topic_keywords = pd.DataFrame(topic_keywords)
df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]
df_topic_keywords

,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,Word 10,Word 11,Word 12,Word 13,Word 14,Word 15,Word 16,Word 17,Word 18,Word 19
Topic 0,eye,machine,platform,victim,operator,perform,resin,activity,time,fall,remove,wear,forklift,hit,safety,head,move,post,person,clean
Topic 1,strike,plate,air,instal,bit,root,son,ladder,use,gun,press,place,connection,screw,attach,drill,assembly,backwards,pole,paper
Topic 2,neck,tech,burn,weld,car,attend,qualified,vehicle,sting,capacitor,insect,unknown,meeting,ice,scrape,move,pass,enter,lateral,front
Topic 3,une,fracture,check,medical,light,start,hospital,service,access,twist,dan,ground,report,turbine,pour,ladder,refer,thumb,activity,welder
Topic 4,arm,technician,bruise,oil,upper,hub,structure,steel,motor,drop,chain,hydraulic,injure,amount,tape,web,spray,tear,bolt,key
Topic 5,cut,hand,cause,cable,leave,head,remove,hit,glove,slip,scratch,plate,door,receive,contact,wire,return,table,finger,piece
Topic 6,step,fall,foot,ankle,slip,cause,knee,leave,hit,floor,walk,blade,trip,leg,left,sprain,result,grind,unit,stand
Topic 7,pain,team,return,hospital,report,continue,site,incident,doctor,ice,treatment,follow,nurse,low,receive,discomfort,examination,time,carry,leader
Topic 8,finger,hand,leave,cut,index,metal,glass,use,roll,report,pull,pinch,left,hit,person,material,pallet,pick,rack,fall
Topic 9,hand,finger,worker,tool,operator,fall,injury,middle,cause,slip,remove,leave,tighten,wrist,pain,arm,bolt,process,position,due
